In [1]:
%%configure -f
{
    "conf": {
        "spark.app.name": "datavirus_delay_analysis"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7933,application_1589299642358_2451,pyspark,idle,Link,Link,
7992,application_1589299642358_2514,pyspark,idle,Link,Link,
7994,application_1589299642358_2517,pyspark,idle,Link,Link,
8008,application_1589299642358_2531,pyspark,idle,Link,Link,
8018,application_1589299642358_2541,pyspark,idle,Link,Link,
8033,application_1589299642358_2553,pyspark,busy,Link,Link,
8037,application_1589299642358_2557,pyspark,idle,Link,Link,
8038,application_1589299642358_2558,pyspark,idle,Link,Link,
8039,application_1589299642358_2559,pyspark,idle,Link,Link,
8042,application_1589299642358_2562,pyspark,idle,Link,Link,


In [2]:
# Initialization
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8082,application_1589299642358_2606,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7933,application_1589299642358_2451,pyspark,idle,Link,Link,
7992,application_1589299642358_2514,pyspark,idle,Link,Link,
7994,application_1589299642358_2517,pyspark,idle,Link,Link,
8008,application_1589299642358_2531,pyspark,idle,Link,Link,
8018,application_1589299642358_2541,pyspark,idle,Link,Link,
8033,application_1589299642358_2553,pyspark,busy,Link,Link,
8037,application_1589299642358_2557,pyspark,idle,Link,Link,
8038,application_1589299642358_2558,pyspark,idle,Link,Link,
8039,application_1589299642358_2559,pyspark,idle,Link,Link,
8042,application_1589299642358_2562,pyspark,idle,Link,Link,


In [64]:
import pyspark.sql.functions as f

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
sbb = spark.read.orc('/data/sbb/orc/istdaten').sample(0.0001).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
sbb.show(1, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+------------+-------------+-------------------------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+----------------+-------------------+------------------+------------+-----------+------------------+-------------+
|betriebstag|fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name                 |produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|bpuic  |haltestellen_name|ankunftszeit    |an_prognose        |an_prognose_status|abfahrtszeit|ab_prognose|ab_prognose_status|durchfahrt_tf|
+-----------+----------------+------------+-------------+-------------------------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+-----------------+----------------+-------------------+------------------+------------+-----------+------------------+-------------+
|03.09.2018 |85:11:11227:003 |85:1

In [77]:
sbb_filtered = (
    sbb
    
    # filter out any trips that we are not interested in
    .where(sbb.zusatzfahrt_tf == 'false')
    .where(sbb.faellt_aus_tf == 'false')
    .where(sbb.durchfahrt_tf == 'false')
    .where(sbb.ankunftszeit.isNotNull())
    .where(sbb.an_prognose.isNotNull())
    .where(sbb.an_prognose_status.isin('REAL', 'PROGNOSE'))
    .where(sbb.betreiber_id.substr(0, 2) == '85')
    
    # convert produkt_id to lowercase
    .withColumn('produkt_id', f.lower(sbb.produkt_id))
    .where(f.col('produkt_id').isin('bus', 'tram', 'zug'))
    
    #
    .withColumn(
        'id',
        f.when(f.col('produkt_id') == 'zug', f.concat_ws(':', sbb.betreiber_id, sbb.linien_id))
        .otherwise(f.concat_ws(':', sbb.betreiber_id, sbb.linien_text))
    )
    
    # calculate delay
    .withColumn('ankunftszeit_ts', f.unix_timestamp(f.col('ankunftszeit'), "dd.MM.yyyy HH:mm").cast('long'))
    .withColumn('an_prognose_ts', f.unix_timestamp(f.col('an_prognose'), "dd.MM.yyyy HH:mm:ss").cast('long'))
    .withColumn('delay', f.col('an_prognose_ts') - f.col('ankunftszeit_ts'))
    .where(f.col('delay').isNotNull())
    
    # sometimes there are more multiple REAL and PROGNOSE values for the exactly same trip on the same day
    # and time. There is no way which one is the closest to the actual arrival time of the train so we 
    # just drop all duplicates and keep one "randomly".
    
    .select(['betriebstag', 'id', 'produkt_id', 'linien_text', 'bpuic', 'ankunftszeit', 'an_prognose_status', 'delay'])
    .dropDuplicates(['betriebstag', 'id', 'bpuic', 'ankunftszeit', 'an_prognose_status'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
sbb_filtered.write.format('orc').save('/user/datavirus/sbb_filtered.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
sbb_filtered = spark.read.orc('/user/datavirus/sbb_filtered.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# sbb = spark.read.orc('/user/datavirus/sbb.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
# remove all stations that are not in 15km radius around zurich 

stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)
stop_ids = [row['stop_id'] for row in stations.collect()]
stop_ids = spark.sparkContext.broadcast(stop_ids)


sbb_zurich = (
    sbb_filtered.
    where(sbb_filtered.bpuic.isin(stop_ids.value))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
sbb_zurich.write.format('orc').save('/user/datavirus/sbb_zurich.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
delays = (
    sbb_zurich
    
    # for each trip we only kept one REAL and one PROGNOSE row
    # now we aggregate them into a single same row
    .groupBy(['id', 'ankunftszeit', 'bpuic'])
    .agg(
        f.first('produkt_id').alias('produkt_id'),
        f.first('linien_text').alias('linien_text'),
        f.first(f.when(f.col('an_prognose_status') == 'REAL', f.col('delay')).otherwise(None)).alias('real_delay'),
        f.first(f.when(f.col('an_prognose_status') == 'PROGNOSE', f.col('delay')).otherwise(None)).alias('prognose_delay'),
    )
    
    # if there is REAL delay we select it, otherwise we take the PROGNOSE delay 
    .withColumn('delay', f.when(f.col('real_delay').isNotNull(), f.col('real_delay')).otherwise(f.col('prognose_delay')))
    .withColumn('is_delayed', f.when(f.col('delay') > 0, 1).otherwise(0))
    
    # create extra columns that contain time without date
    .withColumn('ankunftszeit_minute', f.col('ankunftszeit').substr(12, 5))
    .withColumn('ankunftszeit_hour', f.col('ankunftszeit').substr(12, 2))
    .cache()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
id_probability = (
    delays
    
    .groupBy(['id', 'ankunftszeit_minute', 'bpuic'])
    
    .agg(
        f.first('produkt_id').alias('produkt_id'),
        f.first('linien_text').alias('linien_text'),
        f.mean(delays.is_delayed).alias('id_delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('id_delay_parameter'),
        f.count(delays.delay).alias('id_n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
id_probability.write.format('orc').save('/user/datavirus/id_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
id_probability = spark.read.orc('/user/datavirus/id_probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
line_probability = (
    delays
    
    .groupBy(['id', 'ankunftszeit_hour'])
    
    .agg(
        f.mean(delays.is_delayed).alias('line_delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('line_delay_parameter'),
        f.count(delays.delay).alias('line_n')
    )
    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
line_probability.write.format('orc').save('/user/datavirus/line_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
line_probability = spark.read.orc('/user/datavirus/line_probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
transport_probability = (
    delays
    
    .groupBy(['ankunftszeit_hour', 'produkt_id'])
    
    .agg(
        f.mean(delays.is_delayed).alias('transport_delay_probability'),
        (1.0 / f.mean(f.when(delays.is_delayed == 1, f.when(delays.delay > 30*60, 30*60).otherwise(delays.delay)))).alias('transport_delay_parameter'),
        f.count(delays.delay).alias('transport_n')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
transport_probability.write.format('orc').save('/user/datavirus/transport_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
transport_probability = spark.read.orc('/user/datavirus/transport_probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
transport_probability.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------------+-------------------------+-----------+
|ankunftszeit_hour|produkt_id|transport_delay_probability|transport_delay_parameter|transport_n|
+-----------------+----------+---------------------------+-------------------------+-----------+
|11               |bus       |0.9202279202279202         |0.011397519363432664     |702        |
|22               |bus       |0.9203747072599532         |0.013749912532363025     |427        |
|16               |tram      |0.8207282913165266         |0.011897993990091773     |357        |
|02               |zug       |1.0                        |0.010230179028132993     |4          |
|19               |zug       |0.8148148148148148         |0.01056422569027611      |54         |
+-----------------+----------+---------------------------+-------------------------+-----------+
only showing top 5 rows

In [137]:
full_probability = (
    id_probability
    .withColumn('ankunftszeit_hour', f.col('ankunftszeit_minute').substr(0, 2))
    .join(f.broadcast(line_probability), ['id', 'ankunftszeit_hour'])
    .join(f.broadcast(transport_probability), ['produkt_id', 'ankunftszeit_hour'])
    .select(
        id_probability.id,
        id_probability.ankunftszeit_minute.alias('arrival_time_minute'),
        id_probability.bpuic.alias('station_id'),
        id_probability.produkt_id.alias('transport_type'),
        id_probability.linien_text.alias('line_text'),
        id_probability.id_delay_probability,
        id_probability.id_delay_parameter,
        id_probability.id_n,
        line_probability.line_delay_probability,
        line_probability.line_delay_parameter,
        line_probability.line_n,
        transport_probability.transport_delay_probability,
        transport_probability.transport_delay_parameter,
        transport_probability.transport_n,
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
full_probability.write.format('orc').save('/user/datavirus/full_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
full_probability = spark.read.orc('/user/datavirus/full_probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
full_probability.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+----------+--------------+---------+--------------------+--------------------+----+----------------------+--------------------+------+---------------------------+-------------------------+-----------+
|id         |arrival_time_minute|station_id|transport_type|line_text|id_delay_probability|id_delay_parameter  |id_n|line_delay_probability|line_delay_parameter|line_n|transport_delay_probability|transport_delay_parameter|transport_n|
+-----------+-------------------+----------+--------------+---------+--------------------+--------------------+----+----------------------+--------------------+------+---------------------------+-------------------------+-----------+
|85:11:18991|23:34              |8503312   |zug           |S9       |1.0                 |0.011494252873563218|1   |1.0                   |0.011494252873563218|1     |0.8409090909090909         |0.013663220088626292     |44         |
|85:11:734  |20:16              |8503016   |zug           |IC1  

In [141]:
probability = (
    full_probability
    .select(
        full_probability.id,
        full_probability.arrival_time_minute,
        full_probability.station_id,
        full_probability.transport_type,
        full_probability.line_text,
        (
            f.when(full_probability.id_n > 500, full_probability.id_delay_probability)
            .otherwise(
                f.when(full_probability.line_n > 500, full_probability.line_delay_probability)
                .otherwise(full_probability.transport_delay_probability)
            )
        ).alias('delay_probability'),
        (
            f.when(full_probability.id_n > 500, full_probability.id_delay_parameter)
            .otherwise(
                f.when(full_probability.line_n > 500, full_probability.line_delay_parameter)
                .otherwise(full_probability.transport_delay_parameter)
            )
        ).alias('delay_parameter')  
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
probability.write.format('orc').save('/user/datavirus/probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
probability = spark.read.orc('/user/datavirus/probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
probability.show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+----------+--------------+---------+------------------+--------------------+
|id         |arrival_time_minute|station_id|transport_type|line_text|delay_probability |delay_parameter     |
+-----------+-------------------+----------+--------------+---------+------------------+--------------------+
|85:11:18991|23:34              |8503312   |zug           |S9       |0.8409090909090909|0.013663220088626292|
|85:11:734  |20:16              |8503016   |zug           |IC1      |0.8888888888888888|0.01343408900083963 |
|85:3849:10 |05:21              |8591063   |tram          |10       |0.845360824742268 |0.01970916957096503 |
|85:3849:10 |08:47              |8591406   |tram          |10       |0.8444444444444444|0.011429430784269494|
|85:3849:10 |22:31              |8591073   |tram          |10       |0.8347457627118644|0.013133333333333332|
|85:3849:11 |08:26              |8591299   |tram          |11       |0.8444444444444444|0.011429430784269494|
|85:3849:1